In [1]:
import pandas as pd

df = pd.read_csv('UL-Standards-List.csv')
df = df.dropna()
df.head()

,Standard_Number,Description,Label,Catalog,Edition_Date,Type
0,1,Standard for Flexible Metal Conduit,Ed. 11,UL,"February 16, 2005",ulstd
1,4,Standard for Armored Cable,Ed. 15,UL,"January 15, 2004",ulstd
2,5,Standard for Surface Metal Raceways and Fittings,Ed. 15,UL,"May 24, 2016",ulstd
3,5A,Nonmetallic Surface Raceways and Fittings,Ed. 4,UL,"June 26, 2015",ulcsa
4,5B,Standard for Strut-Type Channel Raceways and F...,Ed. 2,UL,"April 14, 2004",ulstd


In [2]:
# Description to Title Case since there are some descriptions that are all caps
df['Description'] = df['Description'].str.title()



In [18]:
df['Category'] = ''
for index, row in df.iterrows():
    i = row['Description'].find('Standard')
    print(row['Description'][i+8:])
#     if i != -1:
#         df.at[index, 'Category'] = 'Standard'
#         df.at[index, 'Description'] =  row['Description'][i+9:].split(' for ')[1]
#     else:
#         df.at[index, 'Description'] = row['Description']
# df

 For Flexible Metal Conduit
 For Armored Cable
 For Surface Metal Raceways And Fittings
llic Surface Raceways And Fittings
 For Strut-Type Channel Raceways And Fittings
 For Surface Raceways And Fittings For Use With Data, Signal, And Control Circuits
cal Rigid Metal Conduit - Steel
 For Electrical Rigid Metal Conduit - Aluminum And Stainless Steel
 Of Investigation For Pump Tank Extinguishers
ased Agent Fire Extinguishers
 For Fire Tests Of Window Assemblies
 For Tin-Clad Fire Doors
 For Fire Tests Of Door Assemblies
ad Contra Incendio – Conjuntos De Montaje De Puertas Contra Incendio – Método De Prueba De Presión Positiva.
 For Positive Pressure Fire Tests Of Door Assemblies
 For Fire Tests Of Fire-Protective Curtain Assemblies
 For Power-Limited Circuit Cables
 For Sliding Hardware For Standard, Horizontally Mounted Tin-Clad Fire Doors
 For Swinging Hardware For Standard Tin-Clad Fire Doors Mounted Singly And In Pairs
 For Lined Fire Hose And Hose Assemblies
-Use Snap Switches
 For 

In [4]:
# from top2vec import Top2Vec
# model = Top2Vec(documents=df['Description'].tolist())



KeyboardInterrupt

